# BÀI THỰC HÀNH MÁY HỌC NÂNG CAO - TUẦN 4

### Nguyễn Thanh Trúc - 20110342

In [3]:
!pip install -q pytorch-lightning nltk

!gdown 1PD09daNrz8lLm9W5NPVtlLzVnuryN-n4
!unzip data.zip

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip /content/glove.6B.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 59.4 MB/s eta 0:00:00
Downloading...
From: https://drive.google.com/uc?id=1PD09daNrz8lLm9W5NPVtlLzVnuryN-n4
To: /content/data.zip
100% 27.0M/27.0M [00:00<00:00, 74.1MB/s]
Archive:  data.zip
  inflating: IMDB Dataset.csv        
--2023-12-06 12:35:28--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-12-06 12:35:29--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-12-

In [4]:
import pandas as pd
import tqdm
import nltk
import regex as re
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer
import numpy as np
import torch
import torchmetrics
from torch import nn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [130]:
df = pd.read_csv("/content/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [131]:
T = df['review'].str.split(' \n\n---\n\n').str[0]
T = T.str.replace('-',' ').str.replace('[^\w\s]','').str.replace('\n',' ').str.lower()
stop = stopwords.words('english')
T = T.apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))
T = T.apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
df.loc[:, "normalized_review"] = T.to_list()
df.loc[:, "score"] = [1 if label=="positive" else 0 for label in df.sentiment]
df.to_csv("data.csv", index=False)

<ipython-input-131-8960ad0e20a8>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  T = T.str.replace('-',' ').str.replace('[^\w\s]','').str.replace('\n',' ').str.lower()


In [132]:
data = pd.read_csv("data.csv")
data.head()

,review,sentiment,normalized_review,score
0,One of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching oz episode yo...,1
1,A wonderful little production. <br /><br />The...,positive,wonderful little production br br filming tech...,1
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,1
3,Basically there's a family where a little boy ...,negative,basically theres family little boy jake thinks...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...,1


In [133]:
#build the vocab
from nltk import word_tokenize
from functools import reduce
from collections import defaultdict,Counter
cleaned_texts = data['normalized_review'].values.tolist()
word_counter = Counter()

for text in cleaned_texts:
    words = list(word_tokenize(text))
    word_counter.update(words)
print("word num:",len(word_counter))

word2index = {'unk': 0}
for i,word in enumerate(word_counter.keys()):
    word2index[word] = i+1
index2word = {v:k for k,v in word2index.items()}

word num: 151660


In [134]:
index2word[1]

'one'

In [145]:
embeddings_dictionary = dict()
glove_file = open('./glove.6B.100d.txt', encoding="utf8")
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [146]:
#prepare the embedings for lstm model
import torch
embedding_dim = 100
embedding_matrix = torch.zeros((len(word_counter)+1, embedding_dim))
for word, index in word2index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = torch.from_numpy(embedding_vector)

In [147]:
embedding_matrix

tensor([[ 0.0272, -0.1762, -0.1962,  ..., -0.3723, -0.2878, -0.0158],
        [-0.2256,  0.4942,  0.4861,  ..., -0.4574,  0.4965,  0.3491],
        [-0.2096, -0.0578,  0.5238,  ...,  0.0962,  0.3055,  0.6527],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [148]:
#prepare dataloader for lstm model
class TextDataSet(torch.utils.data.Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        if self.labels is not None:
            label = self.labels[idx]
            return text, label
        else:
            return text

def texts2tensor(texts, word2index=word2index, pad_token = 0, max_len = 128):
    indexes_list = [[word2index.get(word,0) for word in word_tokenize(text)] for text in texts]

    max_len = min(max_len,max([len(indexes) for indexes in indexes_list]))
    if max_len > 128:
        raise Exception("max > 128")
    truncated_indexes = [indexes[:max_len] for indexes in indexes_list]
    padded_indexes = [indexes+[0]*(max_len - len(indexes)) for indexes in truncated_indexes]
    return torch.LongTensor(padded_indexes)

def train_collate(batch_inputs):
    texts, labels = zip(*batch_inputs)
    input_tensor = texts2tensor(texts,word2index)
    return input_tensor, torch.LongTensor(labels)


class DataLoader(pl.LightningDataModule):
    def __init__(self, dataframe_path="./data.csv", batch_size=128):

        super(DataLoader, self).__init__()
        self.data = pd.read_csv(dataframe_path)
        self.batch_size = batch_size

    def setup(self, stage=None):
        train_data, val_data = train_test_split(self.data, test_size=0.2, random_state=0)
        train_data, test_data = train_test_split(train_data, test_size=0.25, random_state=0)

        self.train_dataset = TextDataSet(train_data['normalized_review'].values.tolist(),
                                         train_data['score'].values.tolist())
        self.val_dataset = TextDataSet(val_data['normalized_review'].values.tolist(),
                                       val_data['score'].values.tolist())
        self.test_dataset = TextDataSet(test_data['normalized_review'].values.tolist(),
                                       test_data['score'].values.tolist())

    def train_dataloader(self):
        train_loader = torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size,
                                                   shuffle=True, collate_fn=train_collate)
        return train_loader

    def val_dataloader(self):
        val_loader = torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size,
                                                 shuffle=False, collate_fn=train_collate, )
        return val_loader

    def test_dataloader(self):
        test_loader = torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size,
                                  shuffle=False, collate_fn=train_collate)
        return test_loader







dataloader = DataLoader()
dataloader.setup()
train_dataloader = dataloader.train_dataloader()
val_dataloader = dataloader.val_dataloader()

In [149]:
next(iter(val_dataloader))

(tensor([[ 1106, 18758,  1238,  ...,     0,     0,     0],
         [  295,   574,   295,  ...,     0,     0,     0],
         [  126,  1049, 37127,  ...,     0,     0,     0],
         ...,
         [ 9549,  3685,   307,  ...,   630, 45386,  6203],
         [ 4006,   295,    78,  ...,  9763,  7259,  9272],
         [  357,   369,  2397,  ...,     0,     0,     0]]),
 tensor([1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
         1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
         0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
         1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
         1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
         1, 0, 1, 1, 0, 1, 0, 0]))

In [112]:
class LitLSTM(pl.LightningModule):
    def __init__(self, num_classes, embedding, hidden_size=32):
        super(LitLSTM, self).__init__()

        self.embed_size = embedding.shape[1]
        self.embedding = nn.Embedding(embedding.shape[0], embedding.shape[1])
        self.embedding.weight.data.copy_(embedding)
        self.lstm = nn.LSTM(input_size=self.embed_size, hidden_size=hidden_size, num_layers=2, bidirectional=True, batch_first=True)

        self.model = nn.Sequential(nn.Dropout(p=0.7),
                                   nn.ReLU(inplace=True),
                                   nn.Linear(hidden_size, num_classes))

        self.loss = torch.nn.CrossEntropyLoss()
        self.f1 = torchmetrics.classification.BinaryF1Score()
        self.recall = torchmetrics.classification.BinaryRecall()
        self.precision = torchmetrics.classification.BinaryPrecision()

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (final_hidden_state, final_cell_state) = self.lstm(embedded)
        out = self.model(final_hidden_state[-1])
        return out

    def training_step(self, batch, batch_idx):
        texts, labels = batch
        # Forward pass
        outputs = self.forward(texts)
        loss = self.loss(outputs, labels)

        predictions = torch.argmax(outputs, dim=1)
        f1 = self.f1(predictions, labels)
        recall = self.recall(predictions, labels)
        precision = self.precision(predictions, labels)
        self.log_dict({"loss": loss,
                       "f1": f1,
                       "recall": recall,
                       "precision": precision},
                      prog_bar=True)
        return loss

    # define what happens for testing here

    def validation_step(self, batch, batch_idx):
        texts, labels = batch
        # Forward pass
        outputs = self.forward(texts)
        val_loss = self.loss(outputs, labels)

        predictions = torch.argmax(outputs, dim=1)
        val_f1 = self.f1(predictions, labels)
        val_recall = self.recall(predictions, labels)
        val_precision = self.precision(predictions, labels)
        self.log_dict({"val_loss": val_loss,
                       "val_f1": val_f1,
                       "val_recall": val_recall,
                       "val_precision": val_precision},
                      prog_bar=True)

        return val_loss

    def test_step(self, batch, batch_idx):
        texts, labels = batch
        # Forward pass
        outputs = self.forward(texts)
        test_loss = self.loss(outputs, labels)

        predictions = torch.argmax(outputs, dim=1)
        test_f1 = self.f1(predictions, labels)
        test_recall = self.recall(predictions, labels)
        test_precision = self.precision(predictions, labels)
        self.log_dict({"test_loss": test_loss,
                       "test_f1": test_f1,
                       "test_recall": test_recall,
                       "test_precision": test_precision},
                      prog_bar=True)

        return test_loss


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

# seeding for torch, numpy, stdlib random, including DataLoader workers!
#seed_everything(123, workers=True)

early_stopping = EarlyStopping(
    monitor="val_loss",
    mode="min",
    stopping_threshold=1e-5)

model = LitLSTM(hidden_size=64, num_classes=2, embedding=embedding_matrix)
dataloader = DataLoader(batch_size=256)
trainer = Trainer(accelerator="gpu", max_epochs=100, callbacks=[early_stopping])
trainer.fit(model, dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 15.2 M
1 | lstm      | LSTM             | 184 K 
2 | model     | Sequential       | 130   
3 | loss      | CrossEntropyLoss | 0     
4 | f1        | BinaryF1Score    | 0     
5 | recall    | BinaryRecall     | 0     
6 | precision | BinaryPrecision  | 0     
-----------------------------------------------
15.4 M    Trainable params
0         Non-trainable params
15.4 M    Total params
61.402    Total estimated model 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [17]:
df1 = print(trainer.test(ckpt_path="best", datamodule=dataloader))
df1

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=7-step=944.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_0/checkpoints/epoch=7-step=944.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_f1          │    0.8618770241737366     │
│         test_loss         │    0.4913628399372101     │
│      test_precision       │    0.8694709539413452     │
│        test_recall        │    0.8553829789161682     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4913628399372101, 'test_f1': 0.8618770241737366, 'test_recall': 0.8553829789161682, 'test_precision': 0.8694709539413452}]


## **Câu 1**

## Nhận xét:
Mô hình LSTM với glove 100 cho huấn luyện mô hình với batch_size là 128 và số num_layer là 1, dropout rate là 0.9 thì mô hình cho ra được kết quả test_precision tương đối tốt với 0.8694, tuy nhiên thì test loss vẫn còn khá cao (nên giảm chỉ số này xuống) và với mô hình này chỉ cần tinh chỉnh lại số epoch, batch_size hay dropout hoặc num_layer là mô hình có thể cải thiện được hiệu suất mô hình.
Có rất nhiều cách để cải thiện hiệu suất mô hình nhưng em chỉ chọn ra ba cách mà em nghĩ là mang lại hiệu quả trong việc hiệu chỉnh hiệu suất mô hình để đưa ra và dưới đây là ba cách đó.

### Cách để tăng chỉ số test_precision và giảm chỉ số test_loss của glove 100d

### Cách 1: Chỉ tăng mỗi batchsize (từ 128 lên 256) và giữ nguyên num_layer = 1 (cách này thì giúp cho tăng được só lượng epoch lên từ đó cải thiện được hiệu suất mô hình)

In [51]:
df4 = print(trainer.test(ckpt_path="best", datamodule=dataloader))
df4

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_5/checkpoints/epoch=8-step=1062.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_5/checkpoints/epoch=8-step=1062.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_f1          │    0.8529872894287109     │
│         test_loss         │    0.5065613985061646     │
│      test_precision       │    0.8844321370124817     │
│        test_recall        │    0.8247890472412109     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5065613985061646, 'test_f1': 0.8529872894287109, 'test_recall': 0.8247890472412109, 'test_precision': 0.8844321370124817}]


Nhận xét : Đối với cách đầu tiên thì em thử tăng chỉ số batch_size lên và giữ nguyên các giá trị còn lại thì kết quả thu được có vẻ tương đối tốt hơn so với cách trên là test_precision có thể tăng lên khoảng 0.02.

### Cách 2: Tăng batch_size (tăng từ 128 lên 256) và tăng luôn cả num_layer từ 1 lên 4 (vì việc tăng batch_size đã làm cho mô hình tăng lên được accuracy khá tốt cho nên tăng thêm num_layer sẽ giúp cho mô hình học tốt hơn và cải thiện được hiệu suất mô hình)

In [73]:
df5 = print(trainer.test(ckpt_path="best", datamodule=dataloader))
df5

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_8/checkpoints/epoch=6-step=826.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_8/checkpoints/epoch=6-step=826.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_f1          │    0.8668501973152161     │
│         test_loss         │    0.4350094199180603     │
│      test_precision       │    0.8471665978431702     │
│        test_recall        │    0.8884840607643127     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4350094199180603, 'test_f1': 0.8668501973152161, 'test_recall': 0.8884840607643127, 'test_precision': 0.8471665978431702}]


Nhận xét : Đối với cách này thì do là hiệu suất của mô hình ở cách 1 phía trên sau khi tăng cái batch_size lên thì đã có thể cải thiện được hiệu suất mô hình tuy nhiên vẫn cần giảm cái test_loss xuông do đó mà em đã tăng thêm cái num_layer lên để mô hình có thể học được tốt hơn và từ đó test_loss có thể giảm xuống.

### Cách 3: Thay đổi num_dropout bằng cách giảm nó xuống để tăng hiệu suất mô hình (trong quá trình huấn luyện mô hình vì dropout rate có sẵn là 0.9 và em thấy việc dropout này cao và có thể khiến mô hình mất quá nhiều thông tin cho nên em đã giảm xuống còn 0.7 để có thể cải thiện hiệu suất mô hình)

In [113]:
df8 = print(trainer.test(ckpt_path="best", datamodule=dataloader))
df8

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_15/checkpoints/epoch=6-step=826.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_15/checkpoints/epoch=6-step=826.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_f1          │    0.8462207913398743     │
│         test_loss         │    0.4880056083202362     │
│      test_precision       │    0.8949359655380249     │
│        test_recall        │    0.8036400079727173     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4880056083202362, 'test_f1': 0.8462207913398743, 'test_recall': 0.8036400079727173, 'test_precision': 0.8949359655380249}]


Nhận xét : Đối với cách này thì áp dụng cả hai cách trên kết hợp thêm việc thay đổi chỉ số dropout thì mô hình có thể tăng đều cả test_precision, test - f1 và đồng thời cũng giảm được test_loss.

## **Câu 2**

### Dùng glove 200d cho embedding layer

In [97]:
df2 = print(trainer.test(ckpt_path="best", datamodule=dataloader))
df2

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_12/checkpoints/epoch=5-step=708.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_12/checkpoints/epoch=5-step=708.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_f1          │    0.8550756573677063     │
│         test_loss         │    0.46438315510749817    │
│      test_precision       │    0.8994218707084656     │
│        test_recall        │     0.815879762172699     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.46438315510749817, 'test_f1': 0.8550756573677063, 'test_recall': 0.815879762172699, 'test_precision': 0.8994218707084656}]


Nhận xét : Đối với dạng bài này thì khi so sánh kết quả giữa glove 100d, glove 200d và glove 300d thì em nhận thấy rằng kết quả của glove 200d là cho ra kết quả tốt nhất với test_loss là thấp nhất và test_precision lại cao nhất trong giữa ba glove do đó thì em thấy với dạng bài này thì áp dụng glove 200d lại cho ra kết quả tốt nhát.

### Dùng glove 300d cho embedding layer

In [104]:
df3 = print(trainer.test(ckpt_path="best", datamodule=dataloader))
df3

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_13/checkpoints/epoch=6-step=826.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_13/checkpoints/epoch=6-step=826.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_f1          │    0.8606247305870056     │
│         test_loss         │    0.5903210639953613     │
│      test_precision       │    0.8384511470794678     │
│        test_recall        │     0.885302722454071     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5903210639953613, 'test_f1': 0.8606247305870056, 'test_recall': 0.885302722454071, 'test_precision': 0.8384511470794678}]


Nhận xét: Glove 300d thì ngược lại vì nó cho ra hiệu suất mô hình thấp nhất trong cả ba loại.

## **Câu 3**

### Thay LSTM bằng GRU

In [150]:
class LitGRU(pl.LightningModule):
    def __init__(self, num_classes, embedding, hidden_size=32):
        super(LitGRU, self).__init__()

        self.embed_size = embedding.shape[1]
        self.embedding = nn.Embedding(embedding.shape[0], embedding.shape[1])
        self.embedding.weight.data.copy_(embedding)
        self.gru = nn.GRU(input_size=self.embed_size, hidden_size=hidden_size, num_layers=1, bidirectional=True, batch_first=True)

        self.model = nn.Sequential(nn.Dropout(p=0.9),
                                   nn.ReLU(inplace=True),
                                   nn.Linear(hidden_size, num_classes))

        self.loss = torch.nn.CrossEntropyLoss()
        self.f1 = torchmetrics.classification.BinaryF1Score()
        self.recall = torchmetrics.classification.BinaryRecall()
        self.precision = torchmetrics.classification.BinaryPrecision()

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, final_hidden_state = self.gru(embedded)
        out = self.model(final_hidden_state[-1])
        return out

    def training_step(self, batch, batch_idx):
        texts, labels = batch
        # Forward pass
        outputs = self.forward(texts)
        loss = self.loss(outputs, labels)

        predictions = torch.argmax(outputs, dim=1)
        f1 = self.f1(predictions, labels)
        recall = self.recall(predictions, labels)
        precision = self.precision(predictions, labels)
        self.log_dict({"loss": loss,
                       "f1": f1,
                       "recall": recall,
                       "precision": precision},
                      prog_bar=True)
        return loss

    # define what happens for testing here

    def validation_step(self, batch, batch_idx):
        texts, labels = batch
        # Forward pass
        outputs = self.forward(texts)
        val_loss = self.loss(outputs, labels)

        predictions = torch.argmax(outputs, dim=1)
        val_f1 = self.f1(predictions, labels)
        val_recall = self.recall(predictions, labels)
        val_precision = self.precision(predictions, labels)
        self.log_dict({"val_loss": val_loss,
                       "val_f1": val_f1,
                       "val_recall": val_recall,
                       "val_precision": val_precision},
                      prog_bar=True)

        return val_loss

    def test_step(self, batch, batch_idx):
        texts, labels = batch
        # Forward pass
        outputs = self.forward(texts)
        test_loss = self.loss(outputs, labels)

        predictions = torch.argmax(outputs, dim=1)
        test_f1 = self.f1(predictions, labels)
        test_recall = self.recall(predictions, labels)
        test_precision = self.precision(predictions, labels)
        self.log_dict({"test_loss": test_loss,
                       "test_f1": test_f1,
                       "test_recall": test_recall,
                       "test_precision": test_precision},
                      prog_bar=True)

        return test_loss


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

# seeding for torch, numpy, stdlib random, including DataLoader workers!
#seed_everything(123, workers=True)

early_stopping = EarlyStopping(
    monitor="val_loss",
    mode="min",
    stopping_threshold=1e-5)

model = LitGRU(hidden_size=64, num_classes=2, embedding=embedding_matrix)
dataloader = DataLoader(batch_size=256)
trainer = Trainer(accelerator="gpu", max_epochs=100, callbacks=[early_stopping])
trainer.fit(model, dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 15.2 M
1 | gru       | GRU              | 63.7 K
2 | model     | Sequential       | 130   
3 | loss      | CrossEntropyLoss | 0     
4 | f1        | BinaryF1Score    | 0     
5 | recall    | BinaryRecall     | 0     
6 | precision | BinaryPrecision  | 0     
-----------------------------------------------
15.2 M    Trainable params
0         Non-trainable params
15.2 M    Total params
60.920    Total estimated model 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

#### Chạy Glove 100d với batch_size là 256, num_layers = 1, dropout là 0.9

In [144]:
df10 = print(trainer.test(ckpt_path="best", datamodule=dataloader))
df10

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_27/checkpoints/epoch=5-step=708.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_27/checkpoints/epoch=5-step=708.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_f1          │    0.8563002943992615     │
│         test_loss         │    0.42128536105155945    │
│      test_precision       │    0.8469182848930359     │
│        test_recall        │    0.8670852780342102     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.42128536105155945, 'test_f1': 0.8563002943992615, 'test_recall': 0.8670852780342102, 'test_precision': 0.8469182848930359}]


#### Chạy Glove 100d với batch_size là 128, num_layers = 1, dropout là 0.9

In [151]:
df11 = print(trainer.test(ckpt_path="best", datamodule=dataloader))
df11

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_28/checkpoints/epoch=5-step=708.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_28/checkpoints/epoch=5-step=708.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_f1          │    0.8495253920555115     │
│         test_loss         │    0.49370327591896057    │
│      test_precision       │    0.8796843886375427     │
│        test_recall        │    0.8224804401397705     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.49370327591896057, 'test_f1': 0.8495253920555115, 'test_recall': 0.8224804401397705, 'test_precision': 0.8796843886375427}]


Nhận xét : nhìn chung thì mô hình GRU cho ra kết quả tốt hơn mô hình LSTM, nó cải thiện được một số khuyết điểm của LSTM do đó với cùng một input đầu vào, cùng một giá trị batch_size là 128 và num_layer là 1, dropout rate là 0.9 nhưng mà test loss của gru lại thấp hơn so với test loss của lstm và test_precision của gru lại cao hơn nhiều so với lstm.